In [ ]:
from urllib2 import urlopen
import os as os

In [ ]:
## Function Definitions

def StringToDictionary(row_data):
  result = {}
  for param in row_data.split():
    name, value = param.split('=')
    result[name] = value
  return result

def load():
  import gdata.spreadsheet.service
  gd_client = gdata.spreadsheet.service.SpreadsheetsService()
  gd_client.email = USERNAME
  gd_client.password = PASSWD
  gd_client.ProgrammaticLogin()
  return gd_client

def updateIP(ip):
  import time,string
  gd_client = load()
  docs= gd_client.GetSpreadsheetsFeed()
  spreads = []
  for i in docs.entry: spreads.append(i.title.text)
  spread_number = None
  for i,j in enumerate(spreads):
    if j == SPREADSHEET_NAME: spread_number = i
  if spread_number == None:
    return 0

  key = docs.entry[spread_number].id.text.rsplit('/', 1)[1]
  feed = gd_client.GetWorksheetsFeed(key)
  wksht_id = feed.entry[0].id.text.rsplit('/', 1)[1]
  feed = gd_client.GetListFeed(key,wksht_id)
  thetime = time.strftime('%I:%M%p')
  thedate = time.strftime('%m/%d/%y')
  entry = gd_client.InsertRow(StringToDictionary('date='+thedate+' time='+thetime+' ip='+ip),key,wksht_id)
  return 1


In [ ]:
my_ip = urlopen('http://ip.42.pl/raw').read()
print(my_ip)

In [ ]:
home = os.path.expanduser("~")+'/'# trailing slash required!
print(home)

In [ ]:
USERNAME = 'your_username_here' # google/gmail login id
PASSWD = 'your_password_here' # google/gmail login password

In [ ]:
SPREADSHEET_NAME = 'MyHomeIP' # google documents spreadsheet name
BASE_DIR = home # Base Directory to locally save your IP info

IP_WEBSITE = 'www.whatismyip.org'

In [ ]:
## Executed Code

if os.path.exists(BASE_DIR+'CheckIP'): os.remove(BASE_DIR+'CheckIP')
os.system('wget '+IP_WEBSITE+' -t 2 --output-document='+BASE_DIR+'CheckIP')
fh2 = open(BASE_DIR+'CheckIP','r')
ip2 = fh2.read()
fh2.close()

if os.path.exists(BASE_DIR+'CurrentIP'):
  fh1 = open(BASE_DIR+'CurrentIP','r')
  ip1 = fh1.read()
  fh1.close()
else:
  ip1 = ''



if ip1 == ip2: pass
else:
  res = updateIP(ip2)
  if res == 0: raise 'Please Create a Spreadsheet named \''+SPREADSHEET_NAME+'\' in googleDocs.'
  else:
    fh = open(BASE_DIR+'CurrentIP','w')
    fh.write(ip2)
    fh.close()